# Messaging services

- store and pull messages
- manage read receipts

In [ ]:
//| export

import type { Database } from "tinychat/db.ts";
import { Record as Message } from "tinychat/api/types/chat/tinychat/core/message.ts";
import { ChannelView } from "tinychat/api/types/chat/tinychat/server/defs.ts";
import { ServerView } from "tinychat/api/types/chat/tinychat/server/defs.ts";


const get_time_us = (): string => `${new Date().getTime() * 1000}`;

const cleanupChannelView = (view: ChannelView) => {
  return Object.fromEntries(Object.entries(view).filter(([, v]) => v !== null));
}

export class Messaging {
  constructor(protected db: Database) {}

  public markAllMessagesAsRead({ channel, server, user } : { channel: string, server: string, user: string }) {
    this.db
      .prepare(`INSERT OR REPLACE INTO read_receipts (channel, server, user, time_us) VALUES (:channel, :server, :user, :time)`)
      .run({ channel, user, server, time: get_time_us() });
  }

  public getServers({uris, did, viewer }: { uris?: string[] | undefined; did?: string | undefined; viewer?: string | undefined }) : ServerView[] {
    const sql = (where: string = "") => {      
      const w = [viewer ? `c.user = '${viewer}'` : "", where].filter(q => q).join(" AND ").trim();
      const s = !viewer
        ? `SELECT 
        s.uri,
        s.name,
        s.creator,
        json_group_array(
          json_object(
            'id', c.id,
            'name', c.name,
            'server', c.server,
            'latestMessageReceivedTime', c.latest_message_received_time_us
          )
        ) as channels
        FROM servers s
        INNER JOIN channels c ON c.server = s.uri
        ${w ? `WHERE ${w}` : ""}
        GROUP BY s.uri`
        : `SELECT 
        s.uri,
        s.name,
        s.creator,
        json_group_array(
          json_object(
            'id', c.id,
            'name', c.name,
            'server', c.server,
            'lastMessageReadTime', c.last_message_read_time_us,
            'latestMessageReceivedTime', c.latest_message_received_time_us
          )
        ) as channels
        FROM servers s
        INNER JOIN channel_view c ON c.server = s.uri
        ${w ? `WHERE ${w}` : ""}
        GROUP BY s.uri`;
      return this.db.prepare(s);
    }
      
    let results: ServerView[] = [];
      if (uris && uris.length > 0) {
        results = sql(`s.uri IN (${uris.map((u) => `'${u}'`).join(", ")})`).all<ServerView>();
      } else if (did) {
        results = sql(`s.creator = :did`).all<ServerView>({
          did,
        });
      } else {
        results  = sql().all<ServerView>();
      }

      return results.map((rec) => Object.assign(rec, {
        channels: (rec.channels || []).map(cleanupChannelView),
      }));
  }
  
  public receiveMessage(
    { m, uri, sender, time_us }: {
      m: Message;
      uri: string;
      sender: string;
      time_us: string;
    },
  ) {
    this.db.prepare(`
      INSERT INTO messages (uri, channel, server, text, sender, created_at, time_us) VALUES (
        :uri, :channel, :server, :text, :sender, :created_at, :time_us
      )`).run({
      uri,
      channel: m.channel,
      server: m.server,
      text: m.text,
      sender,
      created_at: m.createdAt,
      time_us: time_us,
    });
  }
}

In [ ]:
//| export

import { getDatabase} from "tinychat/db.ts";
import { TID } from "@atproto/common";

class TestMessaging extends Messaging {
  constructor() {
    super(getDatabase({ reset: true }));
  }

  public static server: string = "at://server-1";
  public static user1: string = "did:1";
  public static user2: string = "did:2";
  public static channel1: string = TID.nextStr();
  public static channel2: string = TID.nextStr();

  public user1MessagesChannel1(text: string) {
    this.receiveMessage({
      m: {
        channel: TestMessaging.channel1,
        server: "at://server-1",
        text,
        createdAt: new Date().toISOString(),
      },
      uri: "at://message-1",
      sender: TestMessaging.user1,
      time_us: get_time_us(),
    });
  }

  public static setup(): TestMessaging {
    const service = new TestMessaging();
    // insert 2 test users
    [1, 2].forEach((i) => {
      service.db
        .prepare(
          `
        INSERT INTO users (did, handle, display_name, avatar, description) VALUES (
          :did, :handle, :display_name, :avatar, :description
        )
      `
        )
        .run({
          did: `did:${i}`,
          handle: `user-${i}`,
          display_name: `User ${i}`,
          avatar: `http://google.com/avatar-${i}.jpeg`,
          description: `description ${i}`,
        });
    });
    // create test server
    service.db
      .prepare(
        `
      INSERT INTO servers (uri, name, creator) VALUES (
        :uri, :name, :creator
      )
    `
      )
      .run({
        uri: "at://server-1",
        name: "Test Server",
        creator: "did:1",
      });

    // create memberships for both users
    [1, 2].forEach((i) => {
      service.db
        .prepare(
          `
        INSERT INTO server_memberships (user, server) VALUES (
          :user, :server
        )
      `
        )
        .run({
          user: `did:${i}`,
          server: "at://server-1",
        });
    });

    // setup channels
    [TestMessaging.channel1, TestMessaging.channel2].forEach((c) => {
      service.db
        .prepare(
          `
        INSERT INTO channels (id, name, server) VALUES (
          :id, :name, :server
        )
      `
        )
        .run({
          id: c,
          name: `channel ${c}`,
          server: "at://server-1",
        });
    });

    // set up another chat server
    service.db
      .prepare(
        `
      INSERT INTO servers (uri, name, creator) VALUES (
        :uri, :name, :creator
      )
    `
      )
      .run({
        uri: "at://server-2",
        name: "Test Server 2",
        creator: "did:1",
      });

    // create memberships for both users
    [1, 2].forEach((i) => {
      service.db
        .prepare(
          `
        INSERT INTO server_memberships (user, server) VALUES (
          :user, :server
        )
      `
        )
        .run({
          user: `did:${i}`,
          server: "at://server-2",
        });
    });

    // setup channels
    [1, 2].forEach((i) => {
      service.db
        .prepare(
          `
        INSERT INTO channels (id, name, server) VALUES (
          :id, :name, :server
        )
      `
        )
        .run({
          id: TID.nextStr(),
          name: `channel server 2 ${i}`,
          server: "at://server-2",
        });
    });

    return service;
  }
}


In [ ]:
// import { j } from "@jurassic/jurassic";
// j.initialize("messaging.ipynb");
// await j`
// rebuild this so channels list is pulled in separate subquery instead of a join

// SELECT 
//         s.uri,
//         s.name,
//         s.creator,
//         json_group_array(
//           json_object(
//             'uri', c.uri,
//             'name', c.name,
//             'last_message_read_time_us', c.last_message_read_time_us,
//             'latest_message_received_time_us', c.latest_message_received_time_us
//           )
//         ) as channels
//         FROM servers s
//         INNER JOIN channel_view c ON c.server = s.uri
//         GROUP BY s.uri, s.name, s.creator, c.uri, c.name

// `;


In [ ]:
import { assert, assertEquals } from "asserts";

Deno.test("cleanup ChnanelView", () => {
  assertEquals(
    cleanupChannelView({
      id: "channel-1",
      name: "channel 1",
      server: "at://server-1",
      latestMessageReceivedTime: "123456789",
      lastMessageReadTime: "123456789",
    }),
    {
      id: "channel-1",
      name: "channel 1",
      server: "at://server-1",
      latestMessageReceivedTime: "123456789",
      lastMessageReadTime: "123456789",
    }
  );
  assertEquals(
    cleanupChannelView({
      id: "channel-1",
      name: "channel 1",
      server: "at://server-1",
      // @ts-ignore testing
      latestMessageReceivedTime: null,
      // @ts-ignore testing
      lastMessageReadTime: null,
    }),
    {
      id: "channel-1",
      server: "at://server-1",
      name: "channel 1",
    }
  );
});

Deno.test("get servers includes channel message ts info", () => {
  const messaging = TestMessaging.setup();
  messaging.user1MessagesChannel1("hello world");

  // base case with no viewer set

  let servers = messaging.getServers({
    uris: [TestMessaging.server],
  });

  assertEquals(servers.length, 1, "got 1 server");
  assertEquals(servers[0].channels.length, 2, "got 2 channels");
  assert(servers[0].channels[0].id === TestMessaging.channel1);
  assert(
    servers[0].channels[0].latestMessageReceivedTime, "channel 1 has last message received time set"
  );
  assert(
    typeof servers[0].channels[0].lastMessageReadTime === "undefined",
    "no last message read time data available when viewer para is not set"
  );
  assert(
    typeof servers[0].channels[1].latestMessageReceivedTime === "undefined",
    "channel 2 does not have last message received time set"
  );

  // got a viewer

  servers = messaging.getServers({
    uris: [TestMessaging.server],
    viewer: TestMessaging.user2,
  });

  assertEquals(servers.length, 1);
  assertEquals(servers[0].channels.length, 2, "got 2 channels");
  assert(servers[0].channels[0].id === TestMessaging.channel1);
  assert(
    servers[0].channels[0].latestMessageReceivedTime,
    "channel 1 has last message received time set"
  );
  assert(
    typeof servers[0].channels[0].lastMessageReadTime === "undefined",
    "last message read time data is not set initially"
  );

  // mark all messages as read and recheck

  messaging.markAllMessagesAsRead({
    server: TestMessaging.server,
    channel: TestMessaging.channel1,
    user: TestMessaging.user2,
  });

  servers = messaging.getServers({
    uris: [TestMessaging.server],
    viewer: TestMessaging.user2,
  });

  assertEquals(servers.length, 1);
  assertEquals(servers[0].channels.length, 2, "got 2 channels");
  assert(servers[0].channels[0].id === TestMessaging.channel1);
  assert(
    servers[0].channels[0].lastMessageReadTime,
    "last message read time data is set after marking all messages as read"
  );
});